# Zillow Bestimate
## Wrangle Notebook

In [1]:
# imports
import pandas as pd
import numpy as np
import os
import acquire
import prepare

### Data Acquisition

In [ ]:
df = acquire.get_zillow_data()

In [ ]:
# what do we have to work with?
df.info()

### Data Preparation

In [ ]:
# reset index to parcel_id
df.set_index('parcel_id', drop=True, inplace=True)

In [ ]:
# do we have any whitespace?
df = df.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
# how many null values do I have?
df.isnull().sum()

In [ ]:
# let's drop them
df = df.dropna()

In [ ]:
df.shape

In [ ]:
# do we have any duplicates
df = df.drop_duplicates()
df.shape

In [ ]:
# let's add a column to show the age of the house, not just year built
df['age'] = 2017 - df['yearbuilt']

In [ ]:
df.columns

In [ ]:
# let's convert floats to integers where possible
df['bed'] = df['bed'].astype(int)
df['sqft_calc'] = df['sqft_calc'].astype(int)
df['fips'] = df['fips'].astype(int)
df['yearbuilt'] = df['yearbuilt'].astype(int)
df['appraised_value'] = df['appraised_value'].astype(int)
df['age'] = df['age'].astype(int)

In [ ]:
df.shape

In [ ]:
# calculate upper and lower bounds for bath and remove those outside 1.5 * IQR
bath_25, bath_75 = df['bath'].quantile([0.25, 0.75])
bath_iqr = bath_75 - bath_25
bath_upper = bath_75 + (1.5 * bath_iqr)
bath_lower = bath_25 - (1.5 * bath_iqr)
df = df[df['bath'] > bath_lower]
df = df[df['bath'] < bath_upper]
df.shape

In [ ]:
# calculate upper and lower bounds for bed and remove those outside 1.5 * IQR
bed_25, bed_75 = df['bed'].quantile([0.25, 0.75])
bed_iqr = bed_75 - bed_25
bed_upper = bed_75 + (1.5 * bed_iqr)
bed_lower = bed_25 - (1.5 * bed_iqr)
df = df[df['bed'] > bed_lower]
df = df[df['bed'] < bed_upper]
df.shape

In [ ]:
# calculate upper and lower bounds for square_feet and remove those outside 1.5 * IQR
sqft_25, sqft_75 = df['sqft_calc'].quantile([0.25, 0.75])
sqft_iqr = sqft_75 - sqft_25
limit_upper = sqft_75 + (1.5 * sqft_iqr)
limit_lower = sqft_25 - (1.5 * sqft_iqr)
df = df[df['sqft_calc'] > limit_lower]
df = df[df['sqft_calc'] < limit_upper]
df.shape

In [ ]:
# calculate upper and lower bounds for tax_amount and remove those outside 1.5 * IQR
tax_25, tax_75 = df['tax_amount'].quantile([0.25, 0.75])
tax_iqr = tax_75 - tax_25
tax_upper = tax_75 + (1.5 * tax_iqr)
tax_lower = tax_25 - (1.5 * tax_iqr)
df = df[df['tax_amount'] > tax_lower]
df = df[df['tax_amount'] < tax_upper]
df.shape

In [ ]:
# calculate upper and lower bounds for age and remove those outside 1.5 * IQR
age_25, age_75 = df['age'].quantile([0.25, 0.75])
age_iqr = age_75 - age_25
age_upper = age_75 + (1.5 * age_iqr)
age_lower = age_25 - (1.5 * age_iqr)
df = df[df['age'] > age_lower]
df = df[df['age'] < age_upper]
df.shape

In [ ]:
# repeat for appraised value
app_val_25, app_val_75 = df['appraised_value'].quantile([0.25, 0.75])
app_val_iqr = app_val_75 - app_val_25
upper_value = app_val_75 + (1.5 * app_val_iqr)
lower_value = app_val_25 - (1.5 * app_val_iqr)
df = df[df['appraised_value'] > lower_value]
df = df[df['appraised_value'] < upper_value]
df.shape

In [ ]:
col_list = ['bath', 'bed', 'sqft_calc', 'yearbuilt', 'appraised_value',
       'tax_amount', 'age']

In [2]:
df = prepare.prep_zillow()

In [3]:
df.shape

(23937, 8)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23937 entries, 11721753 to 10713993
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bath             23937 non-null  float64
 1   bed              23937 non-null  int64  
 2   sqft_calc        23937 non-null  int64  
 3   fips             23937 non-null  int64  
 4   yearbuilt        23937 non-null  int64  
 5   appraised_value  23937 non-null  int64  
 6   tax_amount       23937 non-null  float64
 7   age              23937 non-null  int64  
dtypes: float64(2), int64(6)
memory usage: 1.6 MB
